In [1]:
import sys, time
sys.path.append("..")
from motion.longitudinal_controller import LongitudinalController
from motion.lateral_controller import LateralController
from carlasim.carla_client import CarlaClient
from carlasim.ego_car import EgoCar
from planner.stub_slam import StubSLAM
from planner.stub_global_planner import StubGlobalPlanner
from model.waypoint import DistWaypoint
import carla

import gi
gi.require_version('Gst', '1.0')
gi.require_version('GstRtspServer', '1.0')
from gi.repository import Gst, GLib
Gst.init(None)

/workspaces/crawler-poc/notebooks/../carlasim/video_streamer.py:1: PyGIWarning: Gst was imported without specifying a version first. Use gi.require_version('Gst', '1.0') before import to ensure that the right version gets loaded.
  from gi.repository import Gst


[]

In [2]:
StartPosition = ['Town07', -100, 0, 0, 0.0]
global_planner = StubGlobalPlanner()
global_planner.read_mission("./small_mission.dat")
client = CarlaClient(town=StartPosition[0])
ego = EgoCar(client)
ego.set_pose(StartPosition[1], StartPosition[2], StartPosition[3], StartPosition[4])
ego.set_power(0)
ego.set_steering(0)

In [3]:
from carlasim.mqtt_client import MqttClient
from carlasim.ego_car_remote_controller import EgoCarRemoteController
mqtt_client = MqttClient("10.0.1.5", 1883)
manual_control = EgoCarRemoteController(mqtt_client, ego)

In [4]:
world = client.get_world()

for w in global_planner.get_all_poses():    
    world.debug.draw_string(carla.Location(w.x, w.y, 2), 'O', draw_shadow=False,
                                       color=carla.Color(r=255, g=0, b=0), life_time=120.0,
                                       persistent_lines=True)

Lateral Controller

In [5]:
def set_sterr(angle: float) -> None:
    ego.set_steering(angle)
    pass

slam = StubSLAM(ego)

# ctrl = LateralController(
#     vehicle_length=2, 
#     slam_find_current_pose=lambda : slam.estimate_ego_pose(),
#     odometer=lambda : ego.odometer.read(),
#     set_steering_angle=lambda a : set_sterr(a))

ctrl = LateralController(
    vehicle_length=2, 
    slam_find_current_pose=lambda : slam.estimate_ego_pose(),
    odometer=lambda : 10,
    set_steering_angle=lambda a : set_sterr(a))

In [6]:
poses = global_planner.get_all_poses()
ctrl.set_reference_path(poses[0], poses[1])



In [9]:
ego.set_pose(-90, 0, 0, -42)
time.sleep(0.5)
ctrl.loop(1, world)

[1] new heading computed: 40.0


In [26]:
ego.set_brake(0)
ego.set_power(100)
time.sleep(0.5)
ego.set_power(0)
ego.set_brake(1)

In [25]:
ctrl.loop(1, world)

[1] new heading computed: -7.281221218160721
